In [2]:
import sys
sys.path.append('../')
import torch
from transformer.utils import create_tranformer_model
from transformer.utils import load_config, get_dataset, get_tokenizer

## Get Config, dataset, tokenizers
config_file_path = "../config.json"
config = load_config(config_file_path)
dataset = get_dataset(config)
tokenizer_src = get_tokenizer(config, dataset, config['language_source'])
tokenizer_tgt = get_tokenizer(config, dataset, config['language_target'])

# Define the parameters for the transformer model, the dimensions and vocabulary size
vocab_size_src = 10000
vocab_size_tgt = 10000
seq_len = 100
batch_size=1

# Create the transformer model
transformer = create_tranformer_model(
    config,
    vocab_size_src,
    vocab_size_tgt,
)

# Now you can use the transformer model for encoding, decoding, and projecting
# For example, to encode a batch of source sequences:
torch.manual_seed(68) # for reproducible result of random process
src = torch.randint(0, vocab_size_src, (batch_size, seq_len))  # [batch_size, seq_len]
encoder_output = transformer.encode(src)

# And to decode a batch of target sequences:
torch.manual_seed(101) # for reproducible result of random process
tgt = torch.randint(0, vocab_size_tgt, (batch_size, seq_len))  # [batch_size, seq_len]
decoder_output = transformer.decode(tgt, encoder_output)

# And to project the decoder output:
projected_output = transformer.project(decoder_output)

# predicted_token is the indice of the max value in projection_output
_, predicted_tokens_tensors = torch.max(projected_output, dim=1) 

# Explain this
predicted_tokens = predicted_tokens_tensors.squeeze(0)
predicted_tokens = predicted_tokens.detach().cpu().numpy()

## Find the EOS token
for idx, eos in enumerate(predicted_tokens):
    if eos == 3:
        eos_idx = idx
        break

## Get predicted sequence until EOS

predicted_token_sequence = predicted_tokens[:eos_idx]

## Word decode
# Pass the tensor through the target tokenizer
decoded_words = tokenizer_tgt.decode(predicted_token_sequence)

print("Predicted sequence: \n", decoded_words)  

Predicted sequence: 
 pas économie , faire moins Mais politiques un – une , sur Le moins sans tout
